<a href="https://colab.research.google.com/github/Ayushmangal689/Summer_analytics/blob/main/capstone3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install pathway bokeh --quiet # Install necessary libraries (for Colab or Jupyter)

In [15]:
# Import all required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [16]:
# Read CSV dataset
df = pd.read_csv('dataset.csv')
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [17]:
# Combine 'LastUpdatedDate' and 'LastUpdatedTime' columns, convert to datetime, and store in new column 'Timestamp'
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')



In [18]:
# Encode vehicle type into weights
vehicle_weight = {'bike': 0.5 ,'car':1.0,'truck':1.5}
df['VehicleWeight']=df['VehicleType'].map(vehicle_weight)

In [19]:
# Encode traffic conditions numerically
traffic_map={'low': 1,'medium':2, 'high':3}
df['TrafficLevel']=df['TrafficConditionNearby'].map(traffic_map)

In [20]:
# Sort DataFrame by parking lot and timestamp
df = df.sort_values(by=['SystemCodeNumber', 'Timestamp']).reset_index(drop=True)

In [21]:
from math import radians, cos, sin, sqrt, atan2
# Nearby rerouting recommendation logic
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Get mean lat/lon per parking lot
lot_coords = df.groupby('SystemCodeNumber')[['Latitude', 'Longitude']].mean().to_dict('index')

def get_nearby_lots(current_lot, max_distance_km=0.5):
    cur = lot_coords[current_lot]
    nearby = []
    for lot, coords in lot_coords.items():
        if lot == current_lot:
            continue
        distance = haversine(cur['Latitude'], cur['Longitude'], coords['Latitude'], coords['Longitude'])
        if distance <= max_distance_km:
            nearby.append(lot)
    return nearby

# Identify full lots and print reroute suggestions
full_lots = df[df['Occupancy'] >= df['Capacity'] * 0.95]['SystemCodeNumber'].unique()
for lot in full_lots:
    nearby = get_nearby_lots(lot)
    if nearby:
        print(f"\nLot {lot} is full. Consider rerouting to: {nearby}")
    else:
        print(f"\nLot {lot} is full. No nearby alternative.")


Lot BHMBCCMKT01 is full. Consider rerouting to: ['BHMBCCTHL01']

Lot BHMBCCTHL01 is full. Consider rerouting to: ['BHMBCCMKT01']

Lot BHMEURBRD01 is full. No nearby alternative.

Lot BHMMBMMBX01 is full. No nearby alternative.

Lot BHMNCPNST01 is full. Consider rerouting to: ['BHMNCPHST01']

Lot Broad Street is full. No nearby alternative.

Lot Others-CCCPS105a is full. Consider rerouting to: ['Others-CCCPS119a', 'Others-CCCPS135a', 'Others-CCCPS202', 'Others-CCCPS8', 'Others-CCCPS98']
